In [ ]:
# Env/Program Setup

In [ ]:
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/whornsby/neural-style.git

In [ ]:
!pip install -r neural-style/requirements.txt

In [ ]:
!curl 'https://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat' --output imagenet-vgg-verydeep-19.mat

# Utils and Project Code

In [ ]:
'''Display a single image'''
def imgfig(img_file, title, size=10):
  fig = plt.figure(figsize=(size, size))
  img = plt.imread(img_file)
  plt.axis('off')
  plt.title(title)
  plt.imshow(img)


'''Display a list of images vertically'''
def imgshow(img_file, *img_files): 
  n = len(img_files) + 1
  fig, ax = plt.subplots(n,1,figsize=(8, 8*n))

  if n == 1:
    ax.axis('off')
    ax.set_title(img_file)
    im = plt.imread(img_file)
    ax.imshow(im)
  else:
    imgs = (img_file,) + img_files
    for i, imfile in enumerate(imgs):
      ax[i].axis('off')
      ax[i].set_title(imfile)
      im = plt.imread(imfile)
      ax[i].imshow(im)

  plt.show()


'''Display a list of images horizontally'''
def imgshow_h(img_file, *img_files): 
  n = len(img_files) + 1
  fig, ax = plt.subplots(1,n,figsize=(8, 8*n))

  if n == 1:
    ax.axis('off')
    ax.set_title(img_file)
    im = plt.imread(img_file)
    ax.imshow(im)
  else:
    imgs = (img_file,) + img_files
    for i, imfile in enumerate(imgs):
      ax[i].axis('off')
      ax[i].set_title(imfile)
      im = plt.imread(imfile)
      ax[i].imshow(im)

  plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os

def process_upload(img_data, content=False, style=False):
  filename = list(img_data)[0]
  temp_fn = filename.strip().replace(" ", "_")

  if filename != temp_fn:
    os.rename(filename, temp_fn)
    filename = temp_fn
  
  if content and style:
    raise AttributeError("Image must be either content or style, not both")
  if content:
    CONTENT_IMAGE_FN = filename
    print("Content image filename:", filename)
    imgfig(filename, "Content image")
  elif style:
    STYLE_IMAGE_FN = filename
    print("Style image filename:", filename)
    imgfig(filename, "Style image")
  else:
    print("Image saved with filename:", filename)
    imgshow(filename)

# Images

## Upload

In [ ]:
from google.colab import files

content_img = files.upload()
process_upload(content_img, content=True)
CONTENT_IMAGE_FN = list(content_img)[0]

In [ ]:
style_img = files.upload()
process_upload(style_img, style=True)
STYLE_IMAGE_FN = list(style_img)[0]

In [ ]:
#@title Image Overrides
manual_content_image = "marsh8.jpg" #@param {type:"string"}
enable_content_override = True #@param {type:"boolean"}
manual_style_image = "surreal_boat.jpeg" #@param {type:"string"}
enable_style_override = True #@param {type:"boolean"}

if enable_content_override: CONTENT_IMAGE_FN = manual_content_image
if enable_style_override: STYLE_IMAGE_FN = manual_style_image

imgshow_h(CONTENT_IMAGE_FN, STYLE_IMAGE_FN)

## Drive

In [ ]:
MNT_PATH = '/content/drive'
PHOTO_PATH = MNT_PATH + '/MyDrive/StyleTransferPhotos'
CONTENT_PATH = PHOTO_PATH + '/content/'
STYLE_PATH = PHOTO_PATH + '/style/'
OUTPUT_PATH = PHOTO_PATH + '/output/'

In [ ]:
from google.colab import drive
drive.mount(MNT_PATH)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
ROOT_DIR_GDID = '13kelXjrToB3IpbHNj0WmRtP0yRNHcNUg'
CONTENT_DIR_GDID = '11_SKmzv0UqQhxb-rgcLYVsF5sdfUlu6A'
STYLE_DIR_GDID = '1ExIrZyGT2u4brgBIqVzQQk6-j63XP1xu'
OUTPUT_DIR_GDID = '190RRdQlpmipXHztkyK1tF_6WTeA5Ulmo'

def get_file_from_name(name, start_id='root'):
  parents = [start_id]
  while parents:
    pid = parents.pop()
    listed = drive.ListFile({'q': f"'{pid}' in parents"}).GetList()
    for f in listed:
      if file['title'] == name:
        return file
      else: 
        if file['mimeType'] == 'application/vnd.google-apps.folder':
          parents.append(file['id'])


def upload_image(im_file, parent_folder, title=None):
  if title is None:
    title = im_file.split('/')[-1]

  upload_file = drive.CreateFile({'title': title})
  upload_file.SetContentFile(im_file)
  upload_file.Upload()


def download_image(filename, parent_id):
  file_list = drive.ListFile({'q': f'{parent_id} in parents and trashed=false'})


# Style Transfer

## Transfer Job

In [2]:
from transfer import TransferJob

transfer_jobs = []
outputs = []

def run_jobs():
  for job in transfer_jobs:
    output_name = job.Output_name()
    cmd = job.Cmd()
    print("Running Command:")
    print('\n  --'.join(cmd.split('--')))
    !{cmd}
    outputs.append(output_name)
    imgfig(output, "Style Transferred Image - " + output, 20)


IndentationError: expected an indented block (<ipython-input-2-0ed9a513ad6e>, line 40)

## Execute

In [ ]:
#@title  { run: "auto", vertical-output: true }
#@markdown #Command Parameters 
content_image = "/space/washington_space.jpg" #@param {type:"string"}
style_image = "/space/bloom_nebula.jpeg" #@param {type:"string"}
output_dir = "washington_space" #@param {type:"string"}
output_filename = "wa_space_nebula" #@param {type:"string"}
output_width = 1000 #@param {type:"slider", min:500, max:1920, step:10}

#@markdown ---
#@markdown ## Processing

iterations = 1000 #@param {type:"slider", min:500, max:2000, step:100}

# Either 'max' or 'avg' - Default: 'max'
# Original VGG topology uses max pooling, but paper suggests replacing it with avg.
# The outputs are perceptually different, max in general tends to have finer detail style transfer, but could have troubles at lower-freqency detail level
pooling = "max" #@param ["max", "avg"]
preserve_colors = True #@param {type: "boolean"}

#@markdown #### Logging
keep_progress_logs = False #@param {type:"boolean"}
print_iterations =   0 #@param {type:"integer"}
checkpoint_iterations = 200 #@param {type:"integer"}
checkpoint_output = False #@param {type:"boolean"}


#@markdown ---

#TODO: checkpoint_output and checkpoint_iterations
#TODO: style blending

#@markdown ## Tuning
#@markdown ### __Learning__
#@markdown Weights are used in the cost function to balance the similarites of the content image and style image(s) to the generated image.
#@markdown Learning rate is how quickly adjustments are made to lower the cost function.

content_weight_scale = 1 #@param {type:"slider", min:0.1, max:5, step:0.1}
style_weight_scale = 1 #@param {type:"slider", min:0.1, max:5, step:0.1}
learning_rate = 1 #@param {type:"slider", min:0.1, max:2, step:0.1}


#@markdown ### __Style Abstractness__
#@markdown _Float greater than 0.0 - Default: 1.0 (STYLE_LAYER_WEIGHT_EXP)_
#@markdown * Lower values mean __finer details__ of the style will be transfered (also preserves more details of the content image)
#@markdown * Higher values favor the __coarser/larger features__ of the style (can make output look more artsy since the details of the content image can not be represented by the larger stylistic features)
style_abstractness = 0.5 #@param {type:"slider", min:0.0, max:5.0, step:0.1}
style_abstractness_enabled = False #@param {type:"boolean"}

#@markdown ### Content Abstractness
#@markdown _Float between 0.0 and 1.0 - Default: 1.0 (CONTENT_WEIGHT_BLEND)_

#@markdown Conceptually similar to the above, but pertains the the detail/abstracness of the content image
#@markdown * Lower values make the output __MORE abstract__ (reverse of above)
content_abstractness = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1} 
content_abstractness_enabled = False #@param {type:"boolean"}



CONTENT_IMAGE_FN = CONTENT_PATH + content_image
STYLE_IMAGE_FN = STYLE_PATH + style_image
# Script Variables
OUTPUT_WIDTH = output_width # min(output_width, Image.open(CONTENT_IMAGE_FN).size[0])
POOLING = pooling if pooling in ['max', 'avg'] else 'max'

CONTENT_WEIGHT = 5e0 * content_weight_scale  # pulled from neural-style defaults
STYLE_WEIGHT = 5e2 * style_weight_scale  # pulled from neural-style defaults
LEARNING_RATE = learning_rate

STYLE_LAYER_WEIGHT_EXP = style_abstractness if style_abstractness_enabled and 0 < style_abstractness else 1.0
CONTENT_WEIGHT_BLEND = content_abstractness if content_abstractness_enabled and 0 < content_abstractness < 1 else 1.0


from transfer import TransferJob
job = TransferJob(CONTENT_IMAGE_FN, STYLE_IMAGE_FN, output_filename)


if iterations != 1000:
  job.iterations(1000)
if preserve_colors:
  job.preserve_colors()
job.pooling(POOLING)

job.overwrite()

transfer_jobs.append(job)


# if keep_progress_logs: 
#   flags.append("--progress-write")
# if print_iterations > 0:
#   flags.append(f"--print-iterations {print_iterations}")
# if checkpoint_output:
#   flags.append(f"--checkpoint-output {output_filename}_%05d.jpg")
#   flags.append(f"--checkpoint-iterations {checkpoint_iterations}")

# if content_weight_scale != 1:
#   flags.append(f"--content-weight {CONTENT_WEIGHT}")
#   param_string += f"_cWgt{content_weight_scale}"
# if  style_weight_scale != 1:
#   flags.append(f"--style-weight {STYLE_WEIGHT}")
#   param_string += f"_sWgt{style_weight_scale}"
# if learning_rate != 1:
#   flags.append(f"--learning-rate {LEARNING_RATE}")
#   param_string += f"_lr{learning_rate}"

# if style_abstractness_enabled and style_abstractness != 1: 
#   flags.append(f"--style-layer-weight-exp {STYLE_LAYER_WEIGHT_EXP}")
#   param_string += f"_sAb{STYLE_LAYER_WEIGHT_EXP}"
# if content_abstractness_enabled and content_abstractness != 1: 
#   flags.append(f"--content-weight-blend {CONTENT_WEIGHT_BLEND}")
#   param_string += f"_cAb{CONTENT_WEIGHT_BLEND}"


In [ ]:
run_jobs()

In [ ]:
from google.colab import drive
drive.mount(MNT_PATH)

imgshow_h(CONTENT_IMAGE_FN, STYLE_IMAGE_FN)
 
!CMD

imgfig(OUTPUT_IMAGE_FN, "Style Transferred Image - " + OUTPUT_IMAGE_FN, 20)

In [ ]:
# imgshow(CONTENT_IMAGE_FN, STYLE_IMAGE_FN, OUTPUT_IMAGE_FN)
imgfig(OUTPUT_IMAGE_FN, "Style Transferred Image - " + OUTPUT_IMAGE_FN, 20)